<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Mistral_7B_v0_1_McGill_MEDAL_EVALUATOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://medium.com/@frankmorales_91352/fine-tuning-mistral-7b-instruct-v0-1-88b27a194248

In [ ]:
! pip install peft --quiet

!pip install -U flash-attn --no-build-isolation --quiet

#Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

In [2]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)

In [ ]:
!pip install colab-env --quiet

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [ ]:
from huggingface_hub import login

login(
  token=access_token_write, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

In [ ]:
!pip install -U tensorboard-plugin-profile -q

In [ ]:
%rm -rf /content/tensorbard
%mkdir -p /content/tensorbard
%cd /content/tensorbard
!git lfs install
#https://huggingface.co/frankmorales2020/Mistral-7B-v0.1_McGill-MEDAL/tree/main
!git clone https://huggingface.co/frankmorales2020/Mistral-7B-v0.1_McGill-MEDAL

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/tensorbard/Mistral-7B-v0.1_McGill-MEDAL/runs

In [9]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

#peft_model_id = "/content/gdrive/MyDrive/model/Meta-Llama-3-8B-MEDAL-flash-attention-2"

peft_model_id = 'frankmorales2020/Mistral-7B-v0.1_McGill-MEDAL'

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  attn_implementation="flash_attention_2",
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

# load into pipeline
#pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
from datasets import load_dataset
from random import randint


# Load our test dataset
#/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")
#nrec= randint(0, len(eval_dataset))

nrec=6

# Test on sample
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map
prompt =  eval_dataset[nrec]['text']


outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [12]:
ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
ganswer

"'antral follicle count'"

In [19]:
def similarity(ganswer,oanswer):

  qc0=str(ganswer).find('[INST]')

  ganswer=str(ganswer)[1:qc0]
  qc=str(ganswer).find('[/INST]')
  if qc>0:
    ganswer=ganswer[qc+8:len(ganswer)]

  print(f"Generated Answer:\n{ganswer}")

  if ganswer == oanswer:
    print("Match")
    match=1
  else:
    print("NO Match")
    match=0
  return match

In [20]:
print(f"Query:\n{eval_dataset[nrec]['text']}")

oanswer=str(eval_dataset[nrec]['label'])
oanswer=oanswer[2:len(oanswer)-2]
print(f"Original Answer:\n{oanswer}")

ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()

match=similarity(ganswer,oanswer)

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

In [21]:
nrec=6
print(f"Query:\n{eval_dataset[nrec]['text']}")
print(f"Original Answer:\n{eval_dataset[nrec]['label']}")
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

In [22]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
qc=str(ganswer).find('[/INST]')
#print(qc)
if int(qc)<0:
    #print(f"Generated Answer:\n{ganswer}")
    ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
else:
    ganswer=ganswer[qc+8:len(ganswer)]
ganswer

"'antral follicle count'"

In [ ]:
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map

In [ ]:
from tqdm import tqdm

#prompt =  eval_dataset[nrec]['text']


#outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
#                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
#                                  pad_token_id=tokenizer.eos_token_id)


def evaluate(sample):
    prompt =  sample['text']
    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

    ganswer=outputs[0]['generated_text'][len(prompt):].strip()

    #print()
    #print(f"Query:\n{prompt}")

    #print()
    #print(f"Generated Answer original:\n{ganswer}")
    #print()


    qc0=str(ganswer).find("[/INST]")


    if qc0>=0:
       qc2=str(ganswer).find("'")
       #print(qc2)
       ganswer=str(ganswer)[qc2+1:len(ganswer)-1]


    #print(f"Generated Answer corr:\n{ganswer}")

    oanswer=str(sample['label'])
    oanswer=oanswer[2:len(oanswer)-2]

    #print(f"Original Answer:\n{oanswer}")
    if ganswer == oanswer:
        #print()
        #print("Match")
        #print(f"Original Answer:\n{oanswer}")
        #print(f"Generated Answer:\n{ganswer}")
        #print()
        return 1
    else:
        #print()
        #print("no Match")
        #print(f"Original Answer:\n{oanswer}")
        #print(f"Generated Answer:\n{ganswer}")
        #print()
        return 0


#### exceptions TODO
#Original Answer:
#enteropathogenic
#Generated Answer:
#enteropathogenic e coli


success_rate = []
number_of_eval_samples = 1000
# iterate over eval dataset and predict

#for n in tqdm(range(number_of_eval_samples)):
for n in tqdm(range(number_of_eval_samples)):
#    print(f'sample {n}')
    s=eval_dataset[n]
    success_rate.append(evaluate(s))

#for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
#    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

  4%|▍         | 39/1000 [02:27<28:29,  1.78s/it]

In [ ]:
#print(f"Accuracy: {accuracy*100:.2f}%")

print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")

When evaluated on 1000 samples from the evaluation dataset, our model achieved an impressive accuracy of 1000: 26.30%. However, there's room for improvement.